In [4]:
import sqlite3
import pandas as pd
import pickle
import os

In [6]:
TARGET_DECOY_MODEL_DIR = '/home/ubuntu/experiments/dwm-test/target-decoy-models'
METRICS_DB_NAME = '{}/experiment-metrics-for-library-sequences.sqlite'.format(TARGET_DECOY_MODEL_DIR)
SEQUENCE_PEAK_INSTANCES_FILE_NAME = "{}/sequence-peak-instances.pkl".format(TARGET_DECOY_MODEL_DIR)
if os.path.isfile(SEQUENCE_PEAK_INSTANCES_FILE_NAME):
    os.remove(SEQUENCE_PEAK_INSTANCES_FILE_NAME)

In [7]:
# count the number of isolated peaks extracted for each sequence/charge/run combination
# for the training set, we'll only use sequence/charge/run combinations that extracted a single peak
db_conn = sqlite3.connect(METRICS_DB_NAME)
df = pd.read_sql_query("select sequence,charge,file_idx from extracted_metrics;", db_conn)
db_conn.close()
sequence_peak_instances_l = []
for group_name,group_df in df.groupby(['sequence','charge','file_idx'], as_index=False):
    sequence_peak_instances_l.append(tuple(group_name) + (len(group_df),))
sequence_peak_instances_df = pd.DataFrame(sequence_peak_instances_l, columns=['sequence','charge','file_idx','peak_count'])
sequence_peak_instances_df.to_pickle(SEQUENCE_PEAK_INSTANCES_FILE_NAME)
print("{}% of the sequences extracted from all runs have more than one peak".format(round(len(sequence_peak_instances_df[sequence_peak_instances_df.peak_count > 1])/len(sequence_peak_instances_df)*100),1))


70% of the sequences extracted from all runs have more than one peak
